In [1]:
pip install pandas && pip install numpy

Defaulting to user installation because normal site-packages is not writeable
zsh:1: command not found: pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
#import tweepy
import requests
import os
import json
import csv

with open('keys.txt', 'r') as tfile:
    consumer_key = tfile.readline().strip('\n')
    consumer_secret = tfile.readline().strip('\n')
    access_token = tfile.readline().strip('\n')
    access_secret = tfile.readline().strip('\n')
    bearer_token = tfile.readline().strip('\n')


In [4]:
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

In [5]:
def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

In [6]:
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


In [7]:
def set_rules(delete):
    sample_rules = [
        {"value": "covid OR corona OR (vacina causa (hiv OR aids)) OR vachina"},
        #{"value": "vacina OR bolsonaro"},
        
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

In [8]:
def get_params():##bug n to conseguindo colocar mais de um campo por linha, ta impedindo aidiconar mais colunas no csv
    return {"tweet.fields": "lang",
            "user.fields": ["location"],
            "expansions": "author_id",
           }

In [9]:
filename = "tweets"

def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True, params=get_params())
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
        
    #testing
    max_tweet_count = 20
    count = 1
    
    file = open(filename+".csv", 'a')
    writer = csv.writer(file)
    writer.writerow(["tweet", "tweet URL", "author ID"])

    
    for response_line in response.iter_lines():
        #TODO checar e salvar tmb full text tweet (>140 char)
        if response_line:
            decoded_line = response_line.decode('utf-8')
            json_response = json.loads(decoded_line)
            
            #json_response = json.loads(response_line)
            if json_response['data']['text'].startswith('RT @'):
                continue
            if json_response['data']['lang'] != 'pt':#bug ta pegando pt portugal tmb, ta vindo tweet com localizacao na europa
                continue
            #remover tweets resposta
            #remover tweets com excesso de hashtag / emoji
            
            print(json.dumps(json_response, indent=4, sort_keys=True))
            
            tweetText = json_response['data']['text']
            authorId = json_response['data']['author_id']
            tweetId = json_response['data']['id']
            #TODO created_at, author name, media(links, videos?, imagens?)
            linkTweet = "https://twitter.com/user/status/" + str(tweetId)
           
            writer.writerow([tweetText, linkTweet, authorId])
            if count > max_tweet_count:
                break
            count += 1
            
    file.close()

In [10]:
def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)


if __name__ == "__main__":
    main()

{"data": [{"id": "1453966496066011137", "value": "covid OR corona OR (vacina causa (hiv OR aids)) OR vachina"}], "meta": {"sent": "2021-10-29T06:08:28.301Z", "result_count": 1}}
{"meta": {"sent": "2021-10-29T06:08:29.193Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "covid OR corona OR (vacina causa (hiv OR aids)) OR vachina", "id": "1453966954851495936"}], "meta": {"sent": "2021-10-29T06:08:30.113Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200
{
    "data": {
        "author_id": "113660111",
        "id": "1453966961470177291",
        "lang": "pt",
        "text": "@MatheusV_ S\u00e3o fatos que atingem diretamente o mandato e a aprova\u00e7\u00e3o.. \n\nO \u201cfica em ksa\u201d (eu apoiei e fiquei) \n\nN\u00e3o posso reclamar da economia.. vou ter que aguentar calado a crise!\n\nQuem n quis apoiar pq queria trabalhar, tem que aguentar calado a contamina\u00e7\u00e3o .. eh assim a vida!"
    },
    "includes": {
        "users":

{
    "data": {
        "author_id": "1115991386145415169",
        "id": "1453969201262063616",
        "lang": "pt",
        "text": "@frutogostoso e ainda podem pegar covid heuheu"
    },
    "includes": {
        "users": [
            {
                "id": "1115991386145415169",
                "name": "Eme",
                "username": "EmmyPatricio"
            },
            {
                "id": "1036862831478681601",
                "location": "Manaus",
                "name": "1 2 3 de Oliveira 4",
                "username": "frutogostoso"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1453966954851495936",
            "tag": ""
        }
    ]
}
{
    "data": {
        "author_id": "1148599973845635073",
        "id": "1453969245155454984",
        "lang": "pt",
        "text": "e quando o covid era apenas uma piadola de p\u00e1ginas do instagram"
    },
    "includes": {
        "users": [
            {
                "id": "1148